In [14]:
!pip install statsmodels pygam --quiet


In [15]:
# We will need the RBCPath type from the rbclib package to load data from the RBC.
from rbclib import RBCPath

# We'll also want to load some data directly from the filesystem.
from pathlib import Path

# We'll want to load/process some of the data using pandas and numpy.
import pandas as pd
import numpy as np

# This path refers to the repo github.com:ReproBrainChart/PNC_FreeSurfer;
# Subject 1000393599's directory is used as an example.
subject_id = 1000393599
# To browse the repo, use this link:
# https://github.com/ReproBrainChart/PNC_FreeSurfer/tree/main
sub_path = RBCPath(f'rbc://PNC_FreeSurfer/freesurfer/sub-{subject_id}')

# This path refers to a directory:
assert sub_path.is_dir()

# Print each file in the directory:
for file in sub_path.iterdir():
    print(repr(file))

# We can construct new paths by using the `/` operator. This is identical to
# how paths are constructed in the `pathlib` module.
stats_filepath = sub_path / f'sub-{subject_id}_regionsurfacestats.tsv'

# Use pandas to read in the TSV file then display it:

print(f"Loading {stats_filepath} ...")
with stats_filepath.open('r') as f:
    data = pd.read_csv(f, sep='\t')

data

# Participant meta-data is generally located in the BIDS repository for each
# study:
rbcdata_path = Path('/home/jovyan/shared/data/RBC')
train_filepath = rbcdata_path / 'train_participants.tsv'
test_filepath = rbcdata_path / 'test_participants.tsv'

# Load the PNC participants TSV files...
with train_filepath.open('r') as f:
    train_data = pd.read_csv(f, sep='\t')
with test_filepath.open('r') as f:
    test_data = pd.read_csv(f, sep='\t')

# We can also concatenate the two datasets into a single dataset of all
# study participants:
all_data = pd.concat([train_data, test_data])

# Display the full dataframe:
all_data

#step 1
def load_fsdata(participant_id, local_cache_dir=(Path.home() / 'cache')):
    "Loads and returns the dataframe of a PNC participant's FreeSurfer data."

    # Check that the local_cache_dir exists and make it if it doesn't.
    if local_cache_dir is not None:
        local_cache_dir = Path(local_cache_dir)
        local_cache_dir.mkdir(exist_ok=True)
    
    # Make the RBCPath and find the appropriate file:
    pnc_freesurfer_path = RBCPath(
        'rbc://PNC_FreeSurfer/freesurfer',
        # We provide the local_cache_dir to the RBCPath object; all paths made
        # from this object will use the same cache directory.
        local_cache_dir=local_cache_dir)
    participant_path = pnc_freesurfer_path / f'sub-{participant_id}'
    tsv_path = participant_path / f'sub-{participant_id}_regionsurfacestats.tsv'

    # Use pandas to read in the TSV file:
    with tsv_path.open('r') as f:
        data = pd.read_csv(f, sep='\t')

    # Return the loaded data:
    return data


example_participant_id = 1000393599
data = load_fsdata(example_participant_id)

# Display the dataframe we loaded:
data

row_mask = (data['StructName'] == 'Brodmann.1')
data[row_mask]

ba1_surfareas = data.loc[row_mask, 'SurfArea']
ba1_surfarea = sum(ba1_surfareas)

# Show the bilateral surface area for this participant (in square mm):
ba1_surfarea

def load_ba1_surfarea(participant_id):
    """Loads and returns the bilateral Brodmann Area 1 surface area for a PNC
    study participant.
    """
    # First, load the subject's FreeSurfer dataframe:
    data = load_fsdata(participant_id)
    # Next, find the relevant rows:
    row_mask = (data['StructName'] == 'Brodmann.1')
    # Then extract and sum the surface areas:
    ba1_surfareas = data.loc[row_mask, 'SurfArea']
    ba1_surfarea = sum(ba1_surfareas)
    # And return this value:
    return ba1_surfarea

load_ba1_surfarea(example_participant_id)

# First load in surface area data for each participant:
print("Loading surface areas...")     

# We will put the rows in this dictionary of lists as we build the dataframe:
all_vars = {
    'participant_id': [],
    'ba1_surface_area': [],
    'p_factor': []}

# We'll display a progress bar `prog` as we go also:
from ipywidgets import IntProgress
prog = IntProgress(min=0, max=len(all_data))
display(prog)

# Okay, loop through each row of the `all_data` dataframe, which contains both
# training and test subjects, load their BA1 data, and store it in the
# all_vars dictionary.
for (ii, row) in all_data.iterrows():
    # Extract the participant ID and p_factor (which will be NaN for test
    # participants).
    participant_id = row['participant_id']
    p_factor = row['p_factor']
    # Load the surface area for this participant:
    try:
        surf_area = load_ba1_surfarea(participant_id)
    except FileNotFoundError:
        # Some subjects are just missing the file, so we code them as NaN.
        surf_area = np.nan
    # Append the participant ID and their surface area to our dataset:
    all_vars['participant_id'].append(participant_id)
    all_vars['ba1_surface_area'].append(surf_area)
    all_vars['p_factor'].append(p_factor)
    # Increment the progress bar counter:
    prog.value += 1

# Convert train_vars into a dataframe.
all_vars = pd.DataFrame(all_vars)

# Extract the training and test subjects into separate dataframes; the test
# participants can be identified as those having NaN values for their
# p_factor column.
train_vars = all_vars[~np.isnan(all_vars['p_factor'])]
test_vars = all_vars[np.isnan(all_vars['p_factor'])]

# Display the finished dataframe.
all_vars

#regression
# Import the LinearRegression type:
from sklearn.linear_model import LinearRegression

# LinearRegression requires a matrix whose columns are the variables and whose
# final column is the value being predicted (the p_factor for us). We can
# extract these columns straight from the dataframes we generated.
train_matrix = train_vars.loc[:, ['ba1_surface_area', 'p_factor']].values
# We need to exclude rows with NaNs for training:
train_okrows = np.all(~np.isnan(train_matrix), axis=1)
train_matrix = train_matrix[train_okrows]

# Train the regression using the training matrix:
lreg = LinearRegression()
lreg.fit(train_matrix[:, :1], train_matrix[:, 1])

# Display the trained regression parameters:
print("Linear Regression:")
print("  Intercept:", lreg.intercept_)
print("  Slope:", lreg.coef_)

#p factor
# We can apply the trained linear regression object `lreg` to the 1-column
# matrix of ba1_surface_area values in the test_vars dataframe.
test_matrix = test_vars.loc[:, ['ba1_surface_area']].values
test_okrows = np.all(~np.isnan(test_matrix), axis=1)
test_matrix = test_matrix[test_okrows]

# Apply the model:
p_factor_predictions = lreg.predict(test_matrix)

# Display the predictions:
p_factor_predictions

#save and commit
test_data.loc[test_okrows, 'p_factor'] = p_factor_predictions

# Display the resulting test data:
test_data


RBCPath('rbc://PNC_FreeSurfer//home/jovyan/shared/data/RBC/PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_brainmeasures.json')
RBCPath('rbc://PNC_FreeSurfer//home/jovyan/shared/data/RBC/PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_brainmeasures.tsv')
RBCPath('rbc://PNC_FreeSurfer//home/jovyan/shared/data/RBC/PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_freesurfer.tar.xz')
RBCPath('rbc://PNC_FreeSurfer//home/jovyan/shared/data/RBC/PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_fsLR_den-164k.tar.xz')
RBCPath('rbc://PNC_FreeSurfer//home/jovyan/shared/data/RBC/PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_fsaverage.tar.xz')
RBCPath('rbc://PNC_FreeSurfer//home/jovyan/shared/data/RBC/PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_regionsurfacestats.tsv')
Loading rbc://PNC_FreeSurfer/freesurfer/sub-1000393599/sub-1000393599_regionsurfacestats.tsv ...
Loading surface areas...


IntProgress(value=0, max=1601)

Linear Regression:
  Intercept: 0.42976633990916446
  Slope: [-0.0003118]


,participant_id,study,study_site,session_id,wave,age,sex,race,ethnicity,bmi,handedness,participant_education,parent_1_education,parent_2_education,p_factor,internalizing_mcelroy_harmonized_all_samples,externalizing_mcelroy_harmonized_all_samples,attention_mcelroy_harmonized_all_samples,cubids_acquisition_group
0,1000881804,PNC,PNC1,PNC1,1,14.916667,Male,Black,not Hispanic or Latino,21.52,Right,7th Grade,Complete secondary,Complete secondary,-0.429863,0.097355,0.387355,-0.467807,113
1,100527940,PNC,PNC1,PNC1,1,8.250000,Male,Black,not Hispanic or Latino,NaN,Ambidextrous,1st Grade,Complete secondary,Complete primary,-0.416456,0.699062,-0.781881,-0.982040,3
2,1006151876,PNC,PNC1,PNC1,1,21.500000,Female,Other,not Hispanic or Latino,NaN,Right,12th Grade,Complete tertiary,Complete secondary,-0.309821,0.495947,0.806481,-0.832210,1
3,1012530688,PNC,PNC1,PNC1,1,8.750000,Male,Black,not Hispanic or Latino,21.36,Right,2nd Grade,Complete tertiary,Complete secondary,-0.666207,-0.334835,1.277773,0.161110,4
4,1030193285,PNC,PNC1,PNC1,1,18.000000,Male,White,not Hispanic or Latino,22.15,Right,10th Grade,Complete secondary,Complete primary,-0.292984,1.027404,-0.490472,2.014568,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,969649154,PNC,PNC1,PNC1,1,12.333333,Male,White,not Hispanic or Latino,17.38,Right,5th Grade,Complete tertiary,Complete secondary,-0.532445,-0.148520,0.556444,0.024228,1
530,970890500,PNC,PNC1,PNC1,1,18.166667,Female,White,not Hispanic or Latino,30.89,Right,11th Grade,Complete secondary,Complete secondary,-0.267104,0.993806,1.578177,-0.373470,1
531,975856179,PNC,PNC1,PNC1,1,11.000000,Male,White,not Hispanic or Latino,15.67,Right,4th Grade,Complete primary,Complete secondary,-0.462914,-1.026645,-0.582212,1.333857,1
532,984757368,PNC,PNC1,PNC1,1,13.416667,Male,Black,not Hispanic or Latino,16.66,Right,5th Grade,Complete primary,NaN,-0.467279,0.360029,-0.515655,1.509584,114


In [16]:
# NEW. PERFORMING QUANTILE REGRESSION AND GENERAL LINEAR MODEL TO SEE ANY DIFFERENCE

import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from statsmodels.tools import add_constant
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler

# -----------------------------
# 1. Standardize BA1 surface area in training and test
# -----------------------------
scaler = StandardScaler()

# Drop NaNs from training for fitting scaler
train_mask = (~train_vars['ba1_surface_area'].isna()) & (~train_vars['p_factor'].isna())
X_train_raw = train_vars.loc[train_mask, ['ba1_surface_area']]
y_train = train_vars.loc[train_mask, 'p_factor']

# Fit scaler on training BA1
X_train_scaled = scaler.fit_transform(X_train_raw)

# Apply scaling to test set (where BA1 is not NaN)
test_mask = ~test_vars['ba1_surface_area'].isna()
X_test_raw = test_vars.loc[test_mask, ['ba1_surface_area']]
X_test_scaled = scaler.transform(X_test_raw)

# -----------------------------
# 2. GLM (Gaussian)
# -----------------------------
X_train_glm = add_constant(X_train_scaled, has_constant='add')
glm_model = sm.GLM(y_train, X_train_glm, family=sm.families.Gaussian())
glm_results = glm_model.fit()
print("GLM Results:\n", glm_results.summary())

# Predict on test
X_test_glm = add_constant(X_test_scaled, has_constant='add')
glm_predictions = glm_results.predict(X_test_glm)

# -----------------------------
# 3. Quantile Regression (Median)
# -----------------------------
train_df_scaled = pd.DataFrame({
    'p_factor': y_train,
    'ba1_surface_area_scaled': X_train_scaled.flatten()
})

quant_model = smf.quantreg('p_factor ~ ba1_surface_area_scaled', train_df_scaled)
quant_results = quant_model.fit(q=0.5)
print("Quantile Regression Results:\n", quant_results.summary())

# Predict on test
test_df_scaled = pd.DataFrame({
    'ba1_surface_area_scaled': X_test_scaled.flatten()
})
quant_predictions = quant_results.predict(test_df_scaled)

# -----------------------------
# 4. Convert slopes back to raw mm² units
# -----------------------------
ba1_std = train_vars['ba1_surface_area'].std(skipna=True)

glm_slope_scaled = glm_results.params.iloc[1]  # slope in z-score units
quant_slope_scaled = quant_results.params['ba1_surface_area_scaled']  # slope in z-score units

glm_slope_raw = glm_slope_scaled / ba1_std
quant_slope_raw = quant_slope_scaled / ba1_std

print("Slope comparison in raw mm² units:")
print(f"Original Linear Regression slope:   {-0.0003118}")
print(f"GLM slope (converted to raw units): {glm_slope_raw}")
print(f"Quantile slope (converted to raw):  {quant_slope_raw}")

# -----------------------------
# 5. Merge predictions into test_data (safe merge)
# -----------------------------
# Drop old prediction columns if they exist
test_data = test_data.drop(columns=['p_factor_glm', 'p_factor_quantile'], errors='ignore')

# Create prediction DataFrame
pred_df = pd.DataFrame({
    'participant_id': test_vars.loc[test_mask, 'participant_id'].values,
    'p_factor_glm': glm_predictions,
    'p_factor_quantile': quant_predictions.values
})

# Merge on participant_id
test_data = test_data.merge(pred_df, on='participant_id', how='left')

# -----------------------------
# 6. Save results
# -----------------------------
Path("results").mkdir(parents=True, exist_ok=True)
output_file = "results/glm_quantile_comparison.tsv"
test_data.to_csv(output_file, sep='\t', index=False)

print(f"Saved GLM + Quantile Regression predictions to {output_file}")
#TECHNICALLY DON'T SEE NAY DIFFERENCE IN THE RESULTS NETWEEN THE 3 OF THEM.
#THERE IS A WEAK NEGETIVE relationship between BA1 SA and p factor.


GLM Results:
                  Generalized Linear Model Regression Results                  
Dep. Variable:               p_factor   No. Observations:                 1060
Model:                            GLM   Df Residuals:                     1058
Model Family:                Gaussian   Df Model:                            1
Link Function:               Identity   Scale:                         0.85540
Method:                          IRLS   Log-Likelihood:                -1420.3
Date:                Fri, 01 Aug 2025   Deviance:                       905.01
Time:                        19:22:33   Pearson chi2:                     905.
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01390
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4610      0.028    -1